## <font color='darkblue'>Preface</font>
([article source](https://www.kdnuggets.com/2021/12/alternative-feature-selection-methods-machine-learning.html)) <font size='3ptx'><b>You’ve probably done your online searches on "Feature Selection", and you've probably found tons of articles describing the three umbrella terms that group selection methodologies, i.e., "Filter Methods", "Wrapper Methods" and "Embedded Methods".</b></font>

<b>Under the "<font color='darkblue'>Filter Methods</font>", we find statistical tests that select features based on their distributions</b>. These methods are computationally very fast, but in practice they do not render good features for our models. In addition, when we have big datasets, p-values for statistical tests tend to be very small, highlighting as significant tiny differences in distributions, that may not be really important.

<b>The "<font color='darkblue'>Wrapper Methods</font>" category includes greedy algorithms that will try every possible feature combination based on a step forward, step backward, or exhaustive search.</b> For each feature combination, these methods will train a machine learning model, usually with cross-validation, and determine its performance. Thus, wrapper methods are very computationally expensive, and often, impossible to carry out.

<b>The "<font color='darkblue'>Embedded Methods</font>," on the other hand, train a single machine learning model and select features based on the feature importance returned by that model</b>. They tend to work very well in practice and are faster to compute. On the downside, we can’t derive feature importance values from all machine learning models. For example, we can’t derive importance values from nearest neighbours. In addition, co-linearity will affect the coefficient values returned by linear models, or the importance values returned by decision tree based algorithms, which may mask their real importance. Finally, decision tree based algorithms may not perform well in very big feature spaces, and thus, the importance values might be unreliable.

<font color='darkred'><b>Filter Methods are hard to interpret and are not commonly used in practice; Wrapper Methods are computationally expensive and often impossible to carry out; and Embedded Methods are not suitable in every scenario or for every machine learning model. What do we do then? How else can we select predictive features?</b></font>

Fortunately, there are more ways to select features for supervised learning. And I will cover three of them in detail throughout this blog post. For more feature selection methods, check out the online course <b><a href='https://www.udemy.com/course/feature-selection-for-machine-learning/?referralCode=186501DF5D93F48C4F71'>Feature Selection for Machine Learning.</a></b>

### <font color='darkgreen'>Agenda</font>
* <font size='3ptx'><b><a href='#sect1'>Alternative feature selection methods</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Wrapping up</a></b></font>

<a id='sect1'></a>
## <font color='darkblue'>Alternative feature selection methods</font>
* <font size='3ptx'><b><a href='#sect1_1'>Feature shuffling</a></b></font>
* <font size='3ptx'><b><a href='#sect1_2'>Feature performance</a></b></font>
* <font size='3ptx'><b><a href='#sect1_3'>Target mean performance</a></b></font>

<font size='3ptx'><b>In this article, I will describe three algorithms that select features based on their impact on model performance. They are often referred to as "Hybrid Methods" because they share characteristics of Wrapper and Embedded methods.</b></font> Some of these methods rely on training more than one machine learning model, a bit like wrapper methods. Some selection procedures rely on feature importance like Embedded Methods.

But nomenclature aside, these methods have been successfully used in the industry or in data science competitions, and provide additional ways of finding the most predictive features for a certain machine learning model.

Throughout the article, I will lay out the logic and procedure of some of these feature selection methods and show how we can implement them in Python using the open source library Feature-engine. Let's get started.

We will discuss selection by:
* Feature shuffling
* Feature performance
* Target mean performance

<a id='sect1_1'></a>
### <font color='darkgreen'>Feature shuffling</font> ([back](#sect1))
<b>Feature shuffling, or permutation feature importance consists of assigning importance to a feature based on the decrease in a model performance score when the values of a single feature are randomly shuffled.</b> Shuffling the order of the feature values (<font color='brown'>across the rows of the dataset</font>) alters the original relationship between the feature and the target, so the drop in the model performance score is indicative of how much the model depends on that feature.
![feature shuffling](images/galli-feature-shuffling.gif)
<br/>

The procedure works as follows:
1. It trains a machine learning model and determines its performance.
2. It shuffles the order of the values of 1 feature.
3. It makes predictions with the model trained in step 1, and determines the performance.
4. If the performance drops below a threshold, it keeps the feature, otherwise it removes it.
5. It repeats from step 2 until all features are examined.
<br/>

<b>Selection by shuffling features has several advantages. First, we need to train only one machine learning model. The importance is subsequently assigned by shuffling the feature values and making predictions with that model</b>. Second, we can select features for any supervised machine learning model of our choice. Third, we can implement this selection procedure utilizing open source, and we will see how to do this in the coming paragraphs.

<b><font color='green'>Pros:</font></b>
* It only trains one machine learning model, so it is quick.
* It is suitable for any supervised machine learning model.
* It is available in Feature-engine, a Python open source library.
<br/>

On the downside, if two features are correlated, when one of the features is shuffled, the model will still have access to the information through its correlated variable. This may result in a lower importance value for both features, even though they might actually be important. In addition, to select features, we need to define an arbitrary importance threshold below which features will be removed. <b>With higher threshold values, fewer features will be selected. Finally, shuffling features introduces an element of randomness, so for features with borderline importance, that is, importance values close to the threshold, different runs of the algorithm may return different subsets of features.</b>

<b><font color='orange'>Considerations:</font></b>
* Correlations may affect the interpretation of the feature's importance.
* The user needs to define an arbitrary threshold.
* The element of randomness makes the selection procedure non-deterministic.
<br/>

<b>With this in mind, selecting features by feature shuffling is a good feature selection method that focuses on highlighting those variables that directly affect the model performance.</b> We can manually derive the permutation importance with Scikit-learn, and then select those variables that show an importance above a certain threshold. Or we can automate the entire procedure with Feature-engine.

#### <font size='3ptx'>Python Implementation</font>
<b><font size='3ptx'>Let’s see how to carry out selection by feature shuffling with Feature-engine</font></b>. We will use the diabetes dataset that comes with Scikit-learn. First, we load the data:

In [1]:
#!pip install feature_engine

In [2]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from feature_engine.selection import SelectByShuffling

# load dataset
diabetes_X, diabetes_y = load_diabetes(return_X_y=True)
X = pd.DataFrame(diabetes_X)
y = pd.DataFrame(diabetes_y)

We set up the machine learning model we are interested in:

In [3]:
# initialize linear regression estimator
linear_model = LinearRegression()

We will select features based on the drop in the r2 using 3 fold cross-validation:

In [4]:
# initialize the feature selector
tr = SelectByShuffling(estimator=linear_model, scoring="r2", cv=3)

With the method <font color='blue'>fit()</font> the transformer finds the important variables —those that cause a drop in r2 when shuffled. By default, features will be selected if the performance drop is bigger than the mean drop caused by all features.

In [5]:
# fit transformer
tr.fit(X, y)

SelectByShuffling(estimator=LinearRegression(), scoring='r2')

With the method <font color='blue'>transform()</font> we drop the unselected features from the dataset:

In [6]:
Xt1 = tr.transform(X)

We can inspect the individual feature’s importance through one of the transformer’s attributes:

In [7]:
tr.performance_drifts_

{0: -0.023473721086203547,
 1: 0.02264214320305158,
 2: 0.1476297034161565,
 3: 0.051878463192950086,
 4: 0.4416837491272516,
 5: 0.16105759230212935,
 6: -0.006586390190618907,
 7: -0.009084655281897724,
 8: 0.40014559988227094,
 9: -0.020049978497223953}

We can access to the names of the features that will be removed in another attribute:

In [8]:
tr.features_to_drop_

[0, 1, 3, 6, 7, 9]

That’s it, simple. We have a reduced dataframe in `Xt`.

<a id='sect1_2'></a>
### <font color='darkgreen'>Feature performance</font> ([back](#sect1))
<b><font size='3ptx'>A direct way of determining the importance of a feature is to train a machine learning model using solely that feature</font></b>. In this case, the "importance" of the feature is given by the performance score of the model. In other words, how well a model trained on a single feature predicts the target. Poor performance metrics speak of weak or non-predictive features.

The procedure works as follows:
1. It trains a machine learning model for each feature.
2. For each model, it makes predictions and determines model performance.
3. It selects features with performance metrics above a threshold.
<br/>

In this selection procedure, we train one machine learning model per feature. The model uses an individual feature to predict the target variable. Then, we determine the model performance, usually with cross-validation, and select features whose performance falls above a certain threshold.

On one hand, this method is more computationally costly because we would train as many models as features we have in our data set. On the other hand, models trained on a single feature tend to train fairly quickly.

<b>With this method, we can select features for any model that we want, because the importance is given by the performance metric. On the downside, we need to provide an arbitrary threshold for the feature selection</b>. With higher threshold values, we select smaller feature groups. Some threshold values can be fairly intuitive. For example, if the performance metric is the roc-auc, we can select features whose performance is above 0.5. For other metrics, like accuracy, what determines a good value is not so clear.

<b><font color='green'>Pros:</font></b>
* It is suitable for any supervised machine learning model.
* It explores features individually, thus avoiding correlation issues.
* It is available in Feature-engine, a Python open source project.
<br/>

<b><font color='orange'>Considerations:</font></b>
* Training one model per feature can be computationally costly.
* The user needs to define an arbitrary threshold.
* It does not pick up feature interactions.
<br/>

We can implement selection by single feature performance utilizing Feature-engine.

#### <font size='3ptx'>Python Implementation</font>
Let’s load the diabetes dataset from Scikit-learn:

In [9]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from feature_engine.selection import SelectBySingleFeaturePerformance

# load dataset
diabetes_X, diabetes_y = load_diabetes(return_X_y=True)
X = pd.DataFrame(diabetes_X)
y = pd.DataFrame(diabetes_y)

We want to select features whose r2 > 0.01, utilizing a linear regression and using 3 fold cross-validation.

In [10]:
# initialize the feature selector
sel = SelectBySingleFeaturePerformance(
        estimator=LinearRegression(), scoring="r2", cv=3, threshold=0.01)

The transformer uses the method <font color='blue'>fit()</font> to fit 1 model per feature, determine performance, and select the important features.

In [11]:
# fit transformer
sel.fit(X, y)

SelectBySingleFeaturePerformance(estimator=LinearRegression(), scoring='r2',
                                 threshold=0.01)

We can explore the features that will be dropped:

In [12]:
sel.features_to_drop_

[1]

We can also examine each individual feature’s performance:

In [13]:
sel.feature_performance_

{0: 0.029231969375784466,
 1: -0.003738551760264386,
 2: 0.336620809987693,
 3: 0.19219056680145055,
 4: 0.037115559827549806,
 5: 0.017854228256932614,
 6: 0.15153886177526896,
 7: 0.17721609966501747,
 8: 0.3149462084418813,
 9: 0.13876602125792703}

With the method <font color='blue'>transform()</font> we remove the features from the dataset:

In [14]:
# drop variables
Xt2 = sel.transform(X)

And that’s it. Now we have a reduced dataset.

<a id='sect1_3'></a>
### <font color='darkgreen'>Target mean performance</font> ([back](#sect1))
<b><font size='3ptx'>The selection procedure that I will discuss now was introduced in the KDD 2009 data science competition by <a href='http://proceedings.mlr.press/v7/miller09/miller09.pdf'>Miller and co-workers</a></font></b>. The authors do not attribute any name to the technique, but since it uses the mean target value per group of observations as a proxy for predictions, I like to call this technique "Selection by Target Mean Performance."

This selection methodology also assigns an "importance" value to each feature. This importance value is derived from a performance metric. <b>Interestingly, the model does not train any machine learning models. Instead, it uses a much simpler proxy as a prediction.</b>

In a nutshell, the procedure uses the mean target value per category or per interval (<font color='brown'>if the variable is continuous</font>) as a proxy for prediction. <b>With this prediction, it derives a performance metric, like r2, accuracy, or any other metric that assesses a prediction against the truth</b>. 

How does this procedure exactly work?

For categorical variables:
1. It splits the dataframe into a training and a testing set.
2. For every categorical feature, it determines the mean target value per category (<font color='brown'>using the train set</font>).
3. It replaces categories with corresponding target mean values in the test.
4. It determines a performance metric using the encoded features and the target (<font color='brown'>on the test set</font>).
5. It selects features whose performance is above a threshold.
<br/>

For categorical values, the mean value of the target is determined for each category based on the training set. Then, the categories are replaced by the learned values in the test set, and these values are used to determine the performance metric.

For continuous variables, the procedure is fairly similar:
1. It splits the dataframe into a training and a testing set.
2. For every continuous feature, it sorts the values into discrete intervals finding the limits using the train set.
3. It determines the mean target value per interval (<font color='brown'>using a training set</font>).
4. It sorts variables in the test set into the intervals identified in 2.
5. It replaces intervals with corresponding target mean values (<font color='bronw'>using the test set</font>).
6. It determines a performance metric between the encoded feature and the target (<font color='brown'>on the test set</font>).
7. It selects features whose performance is above a threshold.

For continuous variables, the authors first separated the observations into bins, a process otherwise called discretization. They used 1% quantiles. Then they determined the mean value of the target in each bin using the training set and evaluated the performance after replacing the bin values with the target mean in the test set.

<b>This feature selection technique is very simple; it involves taking the mean of the responses for each level</b> (<font color='brown'>category or interval</font>), <b>and comparing these values to the target values to obtain a performance metric. Despite its simplicity, it has a number of advantages.</b>

<b>irst, it does not involve training a machine learning model, so it is incredibly fast to compute. Second, it captures non-linear relationships with the target. Third, it is suitable for categorical variables, unlike the great majority of the existing selection algorithms. It is robust to outliers as these values will be allocated to one of the extreme bins.</b> According to the authors, it offers comparable performance between categorical and numerical variables. And, it is model-agnostic. The features selected by this procedure should, in theory, be suitable for any machine learning model.

<b><font color='green'>Pros:</font></b>
* It is fast because no machine learning model is trained.
* It is suitable for categorical and numerical variables alike.
* It is robust to outliers.
* It captures non-linear relationships between features and the target.
* It is model-agnostic.
<br/>

<b><font color='darkred'>This selection method also presents some limitations. First, for continuous variables, the user needs to define an arbitrary number of intervals in which the values will be sorted. This poses a problem for skewed variables, where most of the values may fall into just one bin. Second, categorical variables with infrequent labels may lead to unreliable results as there are few observations for those categories</font></b>. Therefore, the mean target value per category will be unreliable. In extreme cases, if a category was not present in the training set, we would not have a mean target value to use as a proxy to determine performance.

<b><font color='orange'>Considerations:</font></b>
* It needs tuning of interval numbers for skewed variables.
* Rare categories will offer unreliable performance proxies or make the method impossible to compute.
<br/>

With these considerations in mind, we can select variables based on the target mean performance with Feature-engine.

#### <font size='3ptx'>Python implementation</font>
We will use this method to select variables from the <b><a href='https://www.kaggle.com/c/titanic'>Titanic dataset</a></b>, which has a mix of numerical and categorical variables. When loading the data, I will do some preprocessing to facilitate the demonstration and then separate it into train and test.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from feature_engine.selection import SelectByTargetMeanPerformance

# load data
data = pd.read_csv('../../datas/kaggle_titanic/train.csv')
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [16]:
data.dropna(inplace=True)
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [17]:
# extract cabin letter
data['Cabin'] = data['Cabin'].str[0]

# replace infrequent cabins by N
data['Cabin'] = np.where(data['Cabin'].isin(['T', 'G']), 'N', data['Cabin'])

# cap maximum values
data['Parch'] = np.where(data['Parch']>3, 3, data['Parch'])
data['SibSp'] = np.where(data['SibSp']>3, 3, data['SibSp'])

# cast variables as object to treat as categorical
data[['Pclass','SibSp','Parch']] = data[['Pclass','SibSp','Parch']].astype('O')

In [18]:
# clean nan/missing values
y = data['Survived']
# X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
X = data[['Pclass', 'Sex', 'Age']]
#imp = SimpleImputer(strategy='constant', fill_value='missing')
#imp = imp.fit(X)
#X = imp.transform(X)
X

,Pclass,Sex,Age
1,1,female,38.0
3,1,female,35.0
6,1,male,54.0
10,3,female,4.0
11,1,female,58.0
...,...,...,...
871,1,female,47.0
872,1,male,33.0
879,1,female,56.0
887,1,female,19.0


In [19]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0)

We will select features based on the <b><a href='https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve'>ROC-AUC</a></b> using 2 fold cross-validation. The first thing to note is that Feature-engine allows us to use cross-validation, which is an improvement with respect to the original method described by the authors.

Feature-engine also allows us to decide how we will determine the intervals for numerical variables. We can choose equal frequency or equal width intervals. The authors used 1% quantiles, which is suitable for continuous variables with a fair spread of values, but not often suitable for skewed variables. In this demo, we will separate numerical variables into equal frequency intervals.

Finally, <b>we want to select features for which the ROC-AUC is greater than 0.6.</b>

In [20]:
# Feature-engine automates the selection of 
# categorical and numerical variables
sel = SelectByTargetMeanPerformance(
    variables=None,
    scoring="roc_auc_score",
    threshold=0.6,
    bins=3,
    strategy="equal_frequency",
    cv=2,# cross validation
    random_state=1, # seed for reproducibility
)

With the method <font color='blue'>fit()</font> the transformer:
1. replaces categories by the target mean
2. sorts numerical variables into equal frequency bins
3. replaces bins by the target mean
4. using the target mean encoded variables returns the roc-auc
5. selects features whose roc-auc > 0.6

In [21]:
# find important features
sel.fit(X_train, y_train)

SelectByTargetMeanPerformance(bins=3, cv=2, random_state=1,
                              strategy='equal_frequency', threshold=0.6)

We can explore the ROC-AUC for each feature:

In [22]:
sel.feature_performance_

{'Pclass': 0.5464201683457053,
 'Sex': 0.7666887592788971,
 'Age': 0.6174954157829622}

We can find the features that will be dropped from the data:

In [23]:
sel.features_to_drop_

['Pclass']

With the method <font color='blue'>transform()</font> we drop the features from the data sets:

In [24]:
# remove features
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

Simple. Now we have reduced versions of the train and test sets.

<a id='sect2'></a>
## <font color='darkblue'>Wrapping up</font>
We’ve reached the end of the article. If you made it this far, well done and thank you for reading. If you want to know more about feature selection, including Filter, Wrapper, Embedded and a number of Hybrid methods, check out the online course <b><a href='https://www.udemy.com/course/feature-selection-for-machine-learning/?referralCode=186501DF5D93F48C4F71'>Feature Selection for Machine Learning</a></b>.

For additional courses on machine learning, including Feature Engineering, Hyperparameter Optimization and Model Deployment, visit our <a href='https://www.trainindata.com/'>website</a>.

To implement Filter, Wrapper, Embedded and Hybrid selection methods in Python, check out the selection modules in <a href='https://scikit-learn.org/stable/modules/feature_selection.html'>Scikit-learn</a>, <a href='https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/'>MLXtend</a> and <a href='https://feature-engine.readthedocs.io/en/latest/index.html'>Feature-engine</a>. The libraries come with extensive documentation that will help you understand the underlying methodology.

## <font color='darkblue'>Supplement</font>
* <a href='https://en.wikipedia.org/wiki/Coefficient_of_determination'>Wiki - Coefficient of determination (score $R^2$)</a>